# **Project Name** - FoodLink(Local Food Wastage Management System)

# **Project Summary -**

## Problem Statement
Food wastage is a critical issue in local communities.  
Restaurants, shops, and individuals often have surplus food, while NGOs and shelters struggle to provide enough meals.  
Currently, there is no structured platform to connect these two groups efficiently.

## Proposed Solution
**FoodLink** is designed to bridge this gap.  
It is a database-driven application that connects food providers with receivers, ensuring transparency, reducing wastage, and enabling direct coordination.

## System Design
The system is built around **four key tables**:
1. **Providers** – organizations/individuals offering surplus food.
2. **Receivers** – NGOs, shelters, or individuals requesting food.
3. **Food Listings** – detailed records of food items available (type, meal type, quantity, location).
4. **Claims** – requests made by receivers to claim available food, with status tracking.

## Technologies Used
- **SQLite** – Database for structured storage.
- **Python (sqlite3, pandas)** – Logic and data handling.
- **Streamlit** – Interactive user interface.
- **CSV Export** – For offline access to query results.

## Key Features
- **15 Predefined SQL Queries**: Analyze providers, receivers, claims, and food availability.
- **Dynamic Filters**: By city, provider, food type, and meal type.
- **Provider Contact Details**: Shown for direct coordination.
- **CRUD Operations**: Full add, update, and delete support for all tables (Providers, Receivers, Food Listings, Claims).
- **Downloadable Results**: Export query outputs as CSV.

## Impact
- Reduces food wastage.
- Facilitates direct connection between providers and receivers.
- Transparent record-keeping of food flow.
- Scalable to larger regions with additional features (notifications, dashboards, mobile apps).

## Conclusion
**FoodLink** is a practical solution that combines data management and a user-friendly interface to address a real-world social problem.  
It leverages **Python, SQLite, Pandas, and Streamlit** to provide a complete system that can be deployed and scaled for community use.


# **Problem Statement**

Food wastage is a significant issue, with many households and restaurants discarding surplus food while numerous people struggle with food insecurity. This project aims to develop a Local Food Wastage Management System, where:
* Restaurants and individuals can list surplus food.
* NGOs or individuals in need can claim the food.
* SQL stores available food details and locations.
* A Streamlit app enables interaction, filtering, CRUD operation and visualization. 


# **Buisness Use Case**

* Connecting surplus food providers to those in need through a structured platform.
* Reducing food waste by redistributing excess food efficiently.
* Enhancing accessibility via geolocation features to locate food easily.
* Data analysis on food wastage trends for better decision-making.

# ***Let's Begin !***

In [18]:
from IPython.display import display

# Show full DataFrame without truncating, in a scrollable area
def show_scrollable(df, rows=15, cols=100):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)


In [1]:
#Iporting Libraries
import sqlite3
import pandas as pd

In [8]:
# Connect to the database
conn = sqlite3.connect("C:/Users/omj48/OneDrive/Desktop/Labmentix projects/foodlink/data/foodlink.db")  # if notebook is inside src/
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,providers
1,receivers
2,food_listings
3,claims


## Food Providers & Receivers

### 1. How many food providers and receivers are there in each city?

In [10]:
conn = sqlite3.connect("C:/Users/omj48/OneDrive/Desktop/Labmentix projects/foodlink/data/foodlink.db")  # adjust path if needed

query1 = """
SELECT City,
       COUNT(DISTINCT Provider_ID) AS Total_Providers,
       COUNT(DISTINCT Receiver_ID) AS Total_Receivers
FROM (
    SELECT City, Provider_ID, NULL AS Receiver_ID FROM providers
    UNION ALL
    SELECT City, NULL AS Provider_ID, Receiver_ID FROM receivers
) combined
GROUP BY City
ORDER BY City;
"""

df1 = pd.read_sql(query1, conn)
df1

,City,Total_Providers,Total_Receivers
0,Aaronshire,0,1
1,Adambury,1,0
2,Adamland,0,1
3,Adamsview,1,0
4,Adamsville,1,0
...,...,...,...
1873,Yatesside,1,0
1874,Youngchester,1,0
1875,Zacharyview,0,1
1876,Zimmermanton,1,0


### 2. Which type of food provider (restaurant, grocery store, etc.) contributes the most food?

In [11]:
query2 = """
SELECT Provider_Type,
       SUM(Quantity) AS Total_Quantity
FROM food_listings
GROUP BY Provider_Type
ORDER BY Total_Quantity DESC;
"""

df2 = pd.read_sql(query2, conn)
df2


,Provider_Type,Total_Quantity
0,Restaurant,6923
1,Supermarket,6696
2,Catering Service,6116
3,Grocery Store,6059


Restaurant contributes the most

### 3. What is the contact information of food providers in a specific city?

In [12]:
query_cities = """
SELECT DISTINCT City
FROM (
    SELECT City FROM providers
    UNION
    SELECT City FROM receivers
)
ORDER BY City;
"""

df_cities = pd.read_sql(query_cities, conn)
df_cities


,City
0,Aaronshire
1,Adambury
2,Adamland
3,Adamsview
4,Adamsville
...,...
1873,Yatesside
1874,Youngchester
1875,Zacharyview
1876,Zimmermanton


In [19]:
# Choose the city you want to check
selected_city = "Adambury"  # Change this value to test

query3 = """
SELECT Name, Type, Address, City, Contact
FROM providers
WHERE City = ?
ORDER BY Name;
"""

df3 = pd.read_sql(query3, conn, params=(selected_city,))
df3


,Name,Type,Address,City,Contact
0,Ibarra LLC,Catering Service,"064 Andrea Land Suite 946\nLake Melody, ME 49581",Adambury,6703380260


### Ibarra LLC	Catering Service	064 Andrea Land Suite 946\nLake Melody, ME 49581	Adambury	6703380260

### 4. Which receivers have claimed the most food?

In [21]:
query4 = """
SELECT r.Name AS Receiver_Name,
       r.Type AS Receiver_Type,
       r.City,
       SUM(f.Quantity) AS Total_Quantity_Claimed
FROM claims c
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
JOIN food_listings f ON c.Food_ID = f.Food_ID
WHERE c.Status = 'Completed'
GROUP BY r.Receiver_ID
ORDER BY Total_Quantity_Claimed DESC;
"""

df4 = pd.read_sql(query4, conn)
df4

,Receiver_Name,Receiver_Type,City,Total_Quantity_Claimed
0,Derek Potter,Charity,South Laurachester,99
1,Steven Griffin,NGO,Cruzborough,89
2,Peter Gonzalez,Shelter,Biancaton,83
3,Donna Williams,Charity,Calebview,82
4,Timothy Garrett,NGO,Andersonview,80
...,...,...,...,...
288,Shelly Castillo,Charity,Chenview,2
289,Karen Stafford,Shelter,West Justinberg,2
290,Amanda Kline,Shelter,East Travis,1
291,Janet Newton,Individual,West Amandaport,1


### Derek Potter charity of South Laurachester have claimed the most food

## Food Listings & Availability

### 5. What is the total quantity of food available from all providers?

In [22]:
query5 = """
SELECT SUM(Quantity) AS Total_Food_Quantity
FROM food_listings;
"""

df5 = pd.read_sql(query5, conn)
df5

,Total_Food_Quantity
0,25794


### 25749

### 6. Which city has the highest number of food listings?

In [23]:
query6 = """
SELECT Location AS City,
       COUNT(Food_ID) AS Total_Listings
FROM food_listings
GROUP BY Location
ORDER BY Total_Listings DESC;
"""

df6 = pd.read_sql(query6, conn)
df6

,City,Total_Listings
0,South Kathryn,6
1,New Carol,6
2,Perezport,5
3,Jimmyberg,5
4,East Angela,5
...,...,...
619,Andersonmouth,1
620,Amandashire,1
621,Allenborough,1
622,Alexanderchester,1


### South Kathryn has highest number of food listings

### 7. What are the most commonly available food types?

In [24]:
query7 = """
SELECT Food_Type,
       COUNT(Food_ID) AS Total_Items
FROM food_listings
GROUP BY Food_Type
ORDER BY Total_Items DESC;
"""

df7 = pd.read_sql(query7, conn)
df7

,Food_Type,Total_Items
0,Vegetarian,336
1,Vegan,334
2,Non-Vegetarian,330


### Vegetarian food is the most available food type

## Claims & Distribution

###  8. How many food claims have been made for each food item ?

In [25]:
query8 = """
SELECT f.Food_Name,
       COUNT(c.Claim_ID) AS Total_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY f.Food_Name
ORDER BY Total_Claims DESC;
"""

df8 = pd.read_sql(query8, conn)
df8


,Food_Name,Total_Claims
0,Rice,122
1,Soup,114
2,Dairy,110
3,Fish,108
4,Salad,106
5,Chicken,102
6,Bread,94
7,Pasta,87
8,Vegetables,86
9,Fruits,71


### 9. Which provider has had the highest number of successful food claims?

In [26]:
query9 = """
SELECT p.Name AS Provider_Name,
       p.Type AS Provider_Type,
       COUNT(c.Claim_ID) AS Successful_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
JOIN providers p ON f.Provider_ID = p.Provider_ID
WHERE c.Status = 'Completed'
GROUP BY p.Provider_ID
ORDER BY Successful_Claims DESC;
"""

df9 = pd.read_sql(query9, conn)
df9

,Provider_Name,Provider_Type,Successful_Claims
0,Barry Group,Restaurant,5
1,Miller Inc,Grocery Store,4
2,Butler-Richardson,Grocery Store,4
3,"Barnes, Castro and Curtis",Restaurant,4
4,"Harper, Blake and Alexander",Catering Service,4
...,...,...,...
246,"Black, Collins and Richard",Supermarket,1
247,Miller-Carter,Catering Service,1
248,Nguyen Inc,Supermarket,1
249,Ortiz-Lee,Restaurant,1


### Barry Group provider of type Restaurant has had highest number of food claims that is 5

### 10. What percentage of food claims are completed vs. pending vs. canceled?

In [27]:
query10 = """
SELECT Status,
       COUNT(*) AS Total_Claims,
       ROUND( (COUNT(*) * 100.0) / (SELECT COUNT(*) FROM claims), 2 ) AS Percentage
FROM claims
GROUP BY Status
ORDER BY Percentage DESC;
"""

df10 = pd.read_sql(query10, conn)
df10

,Status,Total_Claims,Percentage
0,Completed,339,33.9
1,Cancelled,336,33.6
2,Pending,325,32.5


## Analysis & Insights

### 11. What is the average quantity of food claimed per receiver?

In [28]:
query11 = """
SELECT r.Name AS Receiver_Name,
       r.Type AS Receiver_Type,
       r.City,
       ROUND(AVG(f.Quantity), 2) AS Avg_Quantity_Claimed
FROM claims c
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
JOIN food_listings f ON c.Food_ID = f.Food_ID
WHERE c.Status = 'Completed'
GROUP BY r.Receiver_ID
ORDER BY Avg_Quantity_Claimed DESC;
"""

df11 = pd.read_sql(query11, conn)
df11

,Receiver_Name,Receiver_Type,City,Avg_Quantity_Claimed
0,Thomas Villanueva,NGO,Gutierrezmouth,50.0
1,Nancy Jones,Shelter,West Bradley,50.0
2,Mark Lewis,NGO,Lisaview,50.0
3,Daniel Burton,Individual,North Matthewhaven,50.0
4,Nancy Silva,Individual,Russellville,50.0
...,...,...,...,...
288,Shelly Castillo,Charity,Chenview,2.0
289,Karen Stafford,Shelter,West Justinberg,2.0
290,Amanda Kline,Shelter,East Travis,1.0
291,Janet Newton,Individual,West Amandaport,1.0


### 12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?

In [29]:
query12 = """
SELECT f.Meal_Type,
       SUM(f.Quantity) AS Total_Quantity_Claimed
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
WHERE c.Status = 'Completed'
GROUP BY f.Meal_Type
ORDER BY Total_Quantity_Claimed DESC;
"""

df12 = pd.read_sql(query12, conn)
df12

,Meal_Type,Total_Quantity_Claimed
0,Breakfast,2423
1,Lunch,2204
2,Dinner,2103
3,Snacks,2043


### Breakfast have been claimed the most

### 13. What is the total quantity of food donated by each provider?

In [30]:
query13 = """
SELECT p.Name AS Provider_Name,
       p.Type AS Provider_Type,
       p.City,
       SUM(f.Quantity) AS Total_Quantity_Donated
FROM food_listings f
JOIN providers p ON f.Provider_ID = p.Provider_ID
GROUP BY p.Provider_ID
ORDER BY Total_Quantity_Donated DESC;
"""

df13 = pd.read_sql(query13, conn)
df13

,Provider_Name,Provider_Type,City,Total_Quantity_Donated
0,Barry Group,Restaurant,South Kathryn,179
1,"Evans, Wright and Mitchell",Catering Service,North Keith,158
2,Smith Group,Restaurant,Jimmyberg,150
3,Nelson LLC,Restaurant,Lake Andrewmouth,142
4,Ruiz-Oneal,Grocery Store,Lake Travis,140
...,...,...,...,...
632,Mcgee PLC,Supermarket,New Hollyfurt,1
633,Brown and Sons,Supermarket,South Christopherborough,1
634,"Martinez, Armstrong and Carroll",Grocery Store,East Lisa,1
635,Gonzales-Moore,Restaurant,Lake Anthonyport,1


### 14. Find the provider(s) whose food is claimed the fastest (based on claim timestamp vs. food listing availability).

In [31]:
query14 = """
SELECT p.Name AS Provider_Name,
       ROUND( CAST(COUNT(c.Claim_ID) AS FLOAT) / COUNT(DISTINCT f.Food_ID), 2 ) AS Avg_Claims_Per_Listing
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
JOIN providers p ON f.Provider_ID = p.Provider_ID
GROUP BY p.Provider_ID
ORDER BY Avg_Claims_Per_Listing DESC;
"""

df14 = pd.read_sql(query14, conn)
df14

,Provider_Name,Avg_Claims_Per_Listing
0,Ramos Group,5.0
1,Taylor Group,4.0
2,Butler-Richardson,4.0
3,Ortega-Webb,4.0
4,"Young, Moran and Morris",4.0
...,...,...
467,Miller Ltd,1.0
468,Brown and Sons,1.0
469,Nguyen Inc,1.0
470,"Clark, Prince and Williams",1.0


### Ramos group's food is claimed the fastest

### 15. List top 5 most popular food items based on total claims quantity.

In [32]:
query15 = """
SELECT f.Food_Name,
       SUM(f.Quantity) AS Total_Quantity_Claimed
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
WHERE c.Status = 'Completed'
GROUP BY f.Food_Name
ORDER BY Total_Quantity_Claimed DESC
LIMIT 5;
"""

df15 = pd.read_sql(query15, conn)
df15

,Food_Name,Total_Quantity_Claimed
0,Bread,1196
1,Soup,1048
2,Rice,1035
3,Salad,1032
4,Dairy,981


# **Conclusion**

**FoodLink** is a practical solution that combines data management and a user-friendly interface to address a real-world social problem.  
It leverages **Python, SQLite, Pandas, and Streamlit** to provide a complete system that can be deployed and scaled for community use